In [1]:
# !pip install numpy transformers pathlib pandas gdown lxml hf_transfer

In [2]:
# !gdown --folder "https://drive.google.com/drive/folders/1wAS0umYohuR53r4sqroxxiG2ab5p5msn"

In [47]:
import os
import math
import random
import numpy as np
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForMaskedLM

In [33]:
TRIPLET_PATH = r"F:\AI-Contest\Genomic_Language_Model\Data\triplet_data.csv"

triplet_df = pd.read_csv(TRIPLET_PATH)
triplet_data = triplet_df.values.tolist()

In [54]:
torch.tensor(triplet_df["seq"][:1])

ValueError: too many dimensions 'str'

In [ ]:
margin = 0.5

anchor = F.normalize(torch.tensor(triplet_df["seq"][:1]), dim=-1)
pos = F.normalize(torch.tensor(triplet_df["positive"][:1]), dim=-1)
neg = F.normalize(torch.tensor(triplet_df["negative"][:1]), dim=-1)

pos_dist = 1 - F.cosine_similarity(anchor, pos, dim=-1)
neg_dist = 1 - F.cosine_similarity(anchor, neg, dim=-1)

loss = F.relu(pos_dist - neg_dist + margin)

loss.mean()

ValueError: too many dimensions 'str'

In [43]:
data = triplet_data[1]
data = [data]
anchor, pos, neg = zip(*data)

In [45]:
anchor

('CAGTAGTGGCATAAACCCAAGGAACAGAGCCAGTGGTACTCCATCCAATGAGCGGGCTCGTCCGGCGTCGGGTATCAGTTCGTTTTTGAATACCTTCGGAATTAGGCAAAATAGCCAGACAGCTTCTTCTTCTGCGGCTCCTGATCAGCGTCTATTCGGCACAACCCCATCTAACTCACATATGAGTGTGGCCATGGAAAGTATCGATACCGCTCCGCAACAGCAGGAACCACGTCTGCATCATCCTATACAAATGCCTCTGTCGGCCCAGTTCCACGTTCATCGCAACTATCAACTCCCCATCTCCATATCTCTCACTGCCCCTACAACCACAGACCACCAGCAATCATCTGCGCATAATTTCGAAGGCAACAATGTTGGTAACGTTCAAGAGTCTTTGAATCAGCGGCAGCCTAACGGTACTAACAATACCACAACCTCGATAATTTCCATGGCGCCAGCGGCTACAACGAGAAATATAGTCGGTGGTGCAGATGGAT',)

In [5]:
triplet_data[:30000]

[['...'],
 ...]

In [6]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

data_path = './Data/'
df = pd.read_csv(data_path + 'test.csv')

# 데이터 최대 길이 확인
max_seq_len = df["seq"].str.len().max()
print(f"✅ Rows = {len(df):,}, Max sequence length = {max_seq_len}")

✅ Rows = 13,711, Max sequence length = 1024


In [8]:
BATCH_SIZE = 64
NUM_WORKERS = 2

MODEL_ID = "InstaDeepAI/nucleotide-transformer-2.5b-multi-species"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
model = model.to(DEVICE).eval()

MODEL_CAP = tokenizer.model_max_length 
EFFECTIVE_MAX_LEN = min(MODEL_CAP, max_seq_len)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.94it/s]


In [9]:
print(MODEL_CAP, max_seq_len)

1000 1024


In [10]:
model

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(4105, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1002, 2560, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-31): 32 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1e-12, elementwise_affine=True)
          )
          (int

In [32]:
model.esm.encoder.layer[32]

IndexError: index 32 is out of range

In [9]:
tokenizer

EsmTokenizer(name_or_path='InstaDeepAI/nucleotide-transformer-v2-100m-multi-species', vocab_size=4107, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
EFFECTIVE_MAX_LEN

1024

In [11]:
class SeqDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.ids  = df["ID"].tolist()
        self.seqs = df["seq"].tolist()
        self.tok  = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return {"ID": self.ids[idx], "seq": self.seqs[idx]}

def collate_fn(batch, tok=tokenizer, max_len=EFFECTIVE_MAX_LEN):
    ids  = [b["ID"] for b in batch]
    seqs = [b["seq"] for b in batch]
    enc  = tok.batch_encode_plus(
        seqs,
        return_tensors="pt",
        padding="longest",          
        truncation=True,
        max_length=max_len
    )
    # attention_mask: pad 토큰이 0
    return {
        "ids": ids,
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"]
    }

In [12]:
dataset = SeqDataset(df, tokenizer, EFFECTIVE_MAX_LEN)
loader  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,
                     num_workers=NUM_WORKERS, collate_fn=collate_fn)
print("✅ Dataloader ready.")

✅ Dataloader ready.


In [ ]:
all_ids = []
all_embs = []
use_amp = (DEVICE == "cuda")

with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attn_mask = batch["attention_mask"].to(DEVICE)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            outs = model(
                input_ids,
                attention_mask=attn_mask,
                encoder_attention_mask=attn_mask,   
                output_hidden_states=True,
            )
            # 마지막 히든스테이트: (B, L, H)
            last_hidden = outs.hidden_states[-1]    # torch.Tensor

            # mask 모양 맞추기: (B, L, 1)
            mask_exp = attn_mask.unsqueeze(-1)      # 1 for valid tokens

            # 패딩 제외 평균: sum(hidden * mask) / sum(mask)
            summed = (last_hidden * mask_exp).sum(dim=1)                    # (B, H)
            counts = mask_exp.sum(dim=1).clamp(min=1)                       # (B, 1)
            seq_emb = summed / counts                                       # (B, H)

        all_ids.extend(batch["ids"])
        all_embs.append(seq_emb.detach().cpu())

emb = torch.vstack(all_embs).float()        # (N, H)
N, H = emb.shape
print(f"✅ Embedding shape = {N} x {H}")

In [ ]:
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

emb_np = emb.numpy()
emb_cols = [f"emb_{i:04d}" for i in range(emb_np.shape[1])]
emb_df = pd.DataFrame(emb_np, columns=emb_cols)

submission = pd.concat([sample_submission['ID'], emb_df], axis=1)
submission.to_csv('baseline_submission.csv', index=False)